# Preparation

In [1]:
!pip install quandl
!pip install arch
!pip install PyPortfolioOpt
!pip install pandas-datareader

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime
import quandl
import math
from statsmodels.stats.diagnostic import het_arch
from arch import arch_model
from pypfopt import EfficientFrontier,objective_functions
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn import metrics
from copy import deepcopy

In [3]:
import sklearn
sklearn.__version__

'1.3.2'

In [4]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [5]:
def modify_date(date):
    return date[:10]

In [6]:
def data_cleaning(data):
    if (data.isna()).any().any():
        missing = np.where(data.isna())
        rows=missing[0]
        cols=missing[1]
        print(rows)
        for row in range(len(rows)):
            col=cols[row]
            if row == len(data)-1:
                data.iloc[row,col]=data.iloc[row-1,col]**2/data.iloc[row-2,col]
            elif row ==0:
                data.iloc[row,col]=data.iloc[row+1,col]**2/data.iloc[row+2,col]
            else:
                data.iloc[row,col]=math.sqrt(data.iloc[row-1,col]*data.iloc[row+1,col])
        print(data)

In [7]:
def get_price_years_ago(data, date,time):
    year_ago=date-datetime.timedelta(days=int(time*365))
    price_year_ago=data[data.index<=year_ago]
    return price_year_ago

In [8]:
def max_drawdown_absolute(returns):
    r = returns.add(1).cumprod()
    dd = r.div(r.cummax()).sub(1)
    mdd = dd.min()
    end = dd.argmin()
    start = r.iloc[:end].argmax()
    return abs(mdd), start, end

In [9]:
def fit_garch(data,p,q):
    lag = max(p,q)
    length =res.shape[0]
    am = arch_model(res, vol='Garch', p=1, o=0, q=1, dist='Normal')
    split_date = one_year_monthly_return.index.values[lag-1]
    results = am.fit(last_obs=split_date)
    forecasts = results.forecast(horizon=1, start = length-lag, method='simulation', simulations=1000)
    sims = forecasts.variance
    std = np.sqrt(np.sum(sims)*12/lag)
    return std

In [10]:
def post_allocation(dfMVOweight):

    dfMVOweight['equal']=1/len(dfMVOweight)
    numofstocks=len(dfMVOweight)
    allocationweight=allocation.loc[numofstocks,:]
    dfMVOweight['proportion']=''
    dfMVOweight['adjusted proportion']=''
    dfMVOweight['direct']=''
    residual=0
    for i in range(numofstocks):
        dfMVOweight.iloc[i,4]=dfMVOweight.iloc[i,1]/np.sum(dfMVOweight['PR_SR'])
        if (dfMVOweight.iloc[i,4]+residual/(numofstocks-i))>allocationweight[0]:
            dfMVOweight.iloc[i,5]=allocationweight[0]
        else:
            dfMVOweight.iloc[i,5]=(dfMVOweight.iloc[i,4]+residual/(numofstocks-i))
        residual+=dfMVOweight.iloc[i,4]-dfMVOweight.iloc[i,5]
    for i in range(int(allocationweight[1])):
        dfMVOweight.iloc[i,6]=allocationweight[0]
    for i in range(int(allocationweight[1]),int(allocationweight[1]+allocationweight[3])):
        dfMVOweight.iloc[i,6]=allocationweight[2]
    return dfMVOweight

In [11]:
def output(weight):
    if stock_type == "S&P100":
        OEX=pdr.get_data_yahoo("^OEX", start=start_swan_date, end=end_swan_date)["Adj Close"]

    elif stock_type == "S&P500":
        OEX=pdr.get_data_yahoo("SPY", start=start_swan_date, end=end_swan_date)["Adj Close"]

    stock_price=pdr.get_data_yahoo(list(weight.index), start=start_swan_date, end=end_swan_date)["Adj Close"]
    OEX.to_csv("oex.csv")
    injected_result = pd.DataFrame(index = stock_price.index )
    S1=np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T
    portfolio_value = stock_price@S1.T
    injected_result["portfolio_value"]=portfolio_value
    S2 = 10000/float(OEX[0])
    injected_result["oex"]=S2*OEX
    return injected_result

# Step 1 - Preselection

In [12]:
# Preselection parameters set up
method = "db"          # or avg or anyother method
stock_type = "S&P100"
end_swan = "Swan10a"         #equals the last swan before the swan we are going to predict
start_swan="Swan4"
start = dt(2019,1,1)
end = dt(2022,1,1)
selected_stock_number = 22     #number of stock preselected
start_swan_date = dt(2022,1,3)
end_swan_date = dt(2023,1,3)
w1 = 0.17
w2 = 0.17
MA_days = 30
N = 7 # This parameter is set to help classification method like K-means

In [13]:
!pip install openpyxl

In [14]:
#read all stocks and their swan beta
if stock_type == "S&P100":
    portfolio_beta = pd.read_excel("All_SwanBeta100.xlsx",index_col=0)
else:
    portfolio_beta = pd.read_excel("All_SwanBeta500.xlsx",index_col=0)

In [15]:
selected_swans_var=portfolio_beta.loc[:,start_swan:end_swan]
portfolio_beta =portfolio_beta[ selected_swans_var.T.isnull().sum()<selected_swans_var.shape[1]-1]

In [16]:
portfolio_beta

,Swan-3,Swan-2,Swan-1,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10a,Swan10b
Ticker,,,,,,,,,,,,,,
CHTR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.583170,0.649030,0.797374,0.939407,1.060138,1.278190
GM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.026711,0.792127,0.190040,1.553517,2.447747,1.934870
KMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540529,1.230239,0.919927,1.592887,-1.141071,-0.541752
GOOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.803565,1.167641,0.916183,0.944312,0.976498,0.875866,1.104899
GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.803565,1.167641,0.870361,0.938163,0.979057,0.880849,1.117074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UNP,0.850695,2.127809,1.993802,1.088334,0.427848,0.418409,0.283910,1.275539,1.334344,1.354688,1.049957,1.188887,-0.200038,0.632258
USB,0.407208,0.375769,0.483245,0.176170,1.187362,1.307068,0.811824,1.672467,0.905422,0.995170,0.906766,1.495656,0.501194,0.825853
WFC,1.388900,1.261192,1.686449,0.558622,1.587780,1.233240,0.300985,1.698834,1.182443,0.995686,0.827634,1.470901,0.571633,1.078228


In [17]:
corr=portfolio_beta.corr()
selected_corr=corr.loc["Swan1":end_swan,"Swan1":end_swan]
corr1={}
for i in range(selected_corr.shape[1]):
    for j in range(i+1,selected_corr.shape[0]):
        selected_corr.iloc[j,i]=np.nan

In [18]:
selected_corr

,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10a
Swan1,1.0,-0.004268,0.111982,0.050073,0.001444,0.101963,0.079685,0.076826,0.037902,-0.051585
Swan2,NaN,1.000000,0.549511,0.335485,0.410861,0.392490,0.033937,0.307858,0.035989,0.412956
Swan3,NaN,NaN,1.000000,0.151778,0.258652,0.435457,0.243705,0.224528,0.103467,0.297051
Swan4,NaN,NaN,NaN,1.000000,-0.032138,0.143125,0.147027,0.225411,0.088824,0.271980
Swan5,NaN,NaN,NaN,NaN,1.000000,0.444474,0.221512,0.208068,0.564402,-0.048352
Swan6,NaN,NaN,NaN,NaN,NaN,1.000000,0.301317,0.482711,0.404329,0.087378
Swan7,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.185835,0.164183,-0.119691
Swan8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.236622,-0.093087
Swan9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,-0.351039
Swan10a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [19]:
avg=selected_corr.mean(axis=1)
weight_swan = avg/sum(avg)
selected_beata=portfolio_beta.loc[:,start_swan:end_swan].fillna(0)
#portfolio_beta["corr_beta"]=selected_beata@weight_swan
#portfolio_beta=portfolio_beta.sort_values(by=["corr_beta"])

In [20]:
portfolio_beta

,Swan-3,Swan-2,Swan-1,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10a,Swan10b
Ticker,,,,,,,,,,,,,,
CHTR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.583170,0.649030,0.797374,0.939407,1.060138,1.278190
GM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.026711,0.792127,0.190040,1.553517,2.447747,1.934870
KMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540529,1.230239,0.919927,1.592887,-1.141071,-0.541752
GOOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.803565,1.167641,0.916183,0.944312,0.976498,0.875866,1.104899
GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.803565,1.167641,0.870361,0.938163,0.979057,0.880849,1.117074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UNP,0.850695,2.127809,1.993802,1.088334,0.427848,0.418409,0.283910,1.275539,1.334344,1.354688,1.049957,1.188887,-0.200038,0.632258
USB,0.407208,0.375769,0.483245,0.176170,1.187362,1.307068,0.811824,1.672467,0.905422,0.995170,0.906766,1.495656,0.501194,0.825853
WFC,1.388900,1.261192,1.686449,0.558622,1.587780,1.233240,0.300985,1.698834,1.182443,0.995686,0.827634,1.470901,0.571633,1.078228


In [21]:
if method == "gamma":
    portfolio_beta["avg"]=portfolio_beta.loc[:,start_swan:end_swan].mean(axis=1)
    portfolio_beta["avg_sd"]=portfolio_beta.loc[:,start_swan:end_swan].mean(axis=1)+portfolio_beta.loc[:,start_swan:end_swan].std(axis=1,ddof=0)
    portfolio_beta=portfolio_beta.sort_values(by=["avg_sd"])
    preselected_stock = portfolio_beta.iloc[:selected_stock_number,:]
elif method =="average":
    portfolio_beta["avg"]=portfolio_beta.loc[:,start_swan:end_swan].mean(axis=1)
    #Select 18 stocks
    portfolio_beta=portfolio_beta.sort_values(by=["avg"])
    preselected_stock = portfolio_beta.iloc[:selected_stock_number,:]

elif method == "K_means":
    selected_beta =portfolio_beta.loc[:,start_swan:end_swan].dropna()
    Y_predic = selected_beta[end_swan].copy()
    Kmean = KMeans(n_clusters=N,random_state=1,n_init=10)#set the random_state to keep modeling the same every run
    Kmean.fit(selected_beta)
    ct = Kmean.cluster_centers_
    labels = Kmean.labels_
    current_dbscore=metrics.davies_bouldin_score(selected_beta, labels)
    print(current_dbscore)
    selected_beta["Group"]=Kmean.labels_
    print(selected_beta['Group'])
    res = ct.mean(axis=1)
    res [res<0]=0

    min_group = np.where(res==min(res))
    for i in range(N):
        Y_predic[selected_beta["Group"] == i] = res [i]
    selected_beta ["avg"]=Y_predic.values
    selected_beta=selected_beta.sort_values(by=["avg"])
    first_group=selected_beta[selected_beta["Group"]==selected_beta["Group"][0]]
    if first_group.shape[0] <10:
        second_group = selected_beta[selected_beta["Group"]==selected_beta["Group"][first_group.shape[0]]]
        preselected_stock = pd.concat([first_group,second_group],axis=0,ignore_index=True)
    else:
        preselected_stock = first_group
        
elif method == "gpt":
    tickers = ['AMZN', 'AAPL', 'MSFT', 'GOOGL', 'NVDA', 'NFLX', 'MA', 'V', 'UNH', 'JPM', 'GS', 'COST', 'HD', 'TGT', 'ADBE', 'BAC', 'LMT', 'BRK-B']
    preselected_stock = portfolio_beta.loc[tickers, start_swan:end_swan]
    preselected_stock["avg"] = preselected_stock.loc[:, start_swan:end_swan].mean(axis=1)
    print("Selected Stocks (GPT):")
    for _, row in preselected_stock.iterrows():
        print(f"Ticker: {row.name}, Average: {row['avg']}")
    print("=" * 50)
    
elif method == "gptLowest8":
    tickers =  ['AMZN', 'AAPL', 'MSFT', 'GOOGL', 'NVDA', 'NFLX', 'MA', 'V', 'UNH', 'JPM', 'GS', 'COST', 'HD', 'TGT', 'ADBE', 'BAC', 'LMT', 'BRK-B']
    preselected_stock = portfolio_beta.loc[tickers, start_swan:end_swan]
    preselected_stock["avg"] = preselected_stock.loc[:, start_swan:end_swan].mean(axis=1)
    print("Selected Stocks (GPT):")
    for _, row in preselected_stock.iterrows():
        print(f"Ticker: {row.name}, Average: {row['avg']}")
    print("=" * 50)

elif method == "db":
    selected_beta =portfolio_beta.loc[:,start_swan:end_swan].dropna()
    Y_predic = selected_beta[end_swan].copy()
    dbscore_tracker = []
    for R in range(1, 1001):  # Loop through random seeds
        Kmean = KMeans(n_clusters=N,random_state=1,n_init=10)#set the random_state to keep modeling the same every run
        Kmean.fit(selected_beta)
        ct = Kmean.cluster_centers_
        labels = Kmean.labels_
        current_dbscore=metrics.davies_bouldin_score(selected_beta, labels)
        print(current_dbscore)
        selected_beta["Group"]=Kmean.labels_
        print(selected_beta['Group'])
        res = ct.mean(axis=1)
        res [res<0]=0
        
        min_group = np.where(res==min(res))
        for i in range(N):
            Y_predic[selected_beta["Group"] == i] = res [i]
            selected_beta ["avg"]=Y_predic.values
            selected_beta=selected_beta.sort_values(by=["avg"])
            first_group=selected_beta[selected_beta["Group"]==selected_beta["Group"][0]]
        if first_group.shape[0] <10:
            second_group = selected_beta[selected_beta["Group"]==selected_beta["Group"][first_group.shape[0]]]
            preselected_stock = pd.concat([first_group,second_group],axis=0,ignore_index=True)
        else:
            preselected_stock = first_group

1.2893048756456593
Ticker
ACN     3
BKNG    3
BLK     6
GS      6
MDLZ    1
       ..
UNP     2
USB     2
WFC     2
WMT     1
XOM     4
Name: Group, Length: 86, dtype: int32
0.976056259506848
Ticker
COP     4
C       0
SLB     4
TMO     2
XOM     4
       ..
MCD     5
VZ      6
GD      1
LLY     6
MSFT    2
Name: Group, Length: 86, dtype: int32
1.0948590116418058
Ticker
SO      0
DHR     2
CVS     0
COF     1
AAPL    2
       ..
TXN     2
WMT     0
EMR     4
COP     5
JNJ     0
Name: Group, Length: 86, dtype: int32
0.8933846414587983
Ticker
CMCSA    1
ABT      1
KO       2
UNH      2
LOW      1
        ..
BLK      4
WBA      1
LLY      2
MO       2
GE       0
Name: Group, Length: 86, dtype: int32
1.0032155417425013
Ticker
TGT     0
IBM     0
INTC    0
PFE     0
SPG     6
       ..
AIG     5
EMR     5
T       0
BLK     4
ACN     1
Name: Group, Length: 86, dtype: int32
0.9806973640757002
Ticker
VZ      2
BLK     0
MO      2
NVDA    6
HD      6
       ..
CL      2
C       0
MDT     2
MET 

In [22]:
# preselected_stock.to_csv('temp.csv')

In [23]:
#portfolio_beta["avg"]=portfolio_beta.loc[:,start_swan:end_swan]*weight

In [24]:
preselected_stock["PR"]=1-preselected_stock["avg"]
preselected_stock.head(selected_stock_number)

/tmp/ipykernel_671/2651548351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preselected_stock["PR"]=1-preselected_stock["avg"]


,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10a,Group,avg,PR
Ticker,,,,,,,,,,
ORCL,1.098732,0.741279,0.833162,0.841817,0.693684,0.643560,1.161396,4,0.527420,0.472580
TMO,0.705951,0.957413,1.434218,0.888224,0.784559,0.786399,1.338748,4,0.527420,0.472580
ABT,0.702188,0.474696,0.311079,1.073150,0.431921,0.941927,1.156495,4,0.527420,0.472580
BIIB,1.431911,0.149965,0.925099,2.333501,1.054590,0.668161,1.232137,4,0.685433,0.314567
CMCSA,1.470030,0.996273,1.036741,1.221346,0.301836,0.814080,0.616578,4,0.685433,0.314567
PFE,0.820699,0.654960,0.746497,0.778217,0.441324,0.677535,1.115363,4,0.685433,0.314567
T,1.384176,0.551527,0.419086,0.666460,0.908832,0.962824,0.551059,4,0.685433,0.314567
DHR,0.720942,0.802835,1.280016,0.453165,0.631525,0.822711,1.322608,4,1.146634,-0.146634
QCOM,1.043829,0.641135,0.982422,1.165628,1.269623,0.964206,1.335199,4,1.247040,-0.247040


# Step 2 - Download data and adjuste price based on Swan Beta

In [211]:
#next line is used to input preselected stock from excel, if you are going to preselect by your self, use this blocks and note next one.
#selected_stock_list =  pd.read_excel("19_stickers.xlsx",header = None).values[:,0]
#swan_beta=pd.read_excel("19_stickers.xlsx",header = None,index_col=0).iloc[:,0]

In [25]:
#Set the paramter to download tradeinfo from Yahoo Finance
selected_stock=preselected_stock
selected_stock_list=selected_stock.index.values
swan_beta=preselected_stock.loc[selected_stock_list,"avg"]

In [26]:
#download data from yahoo finance
yahooDataFrame= pdr.get_data_yahoo(list(selected_stock_list), start=start, end=end)["Adj Close"]

[*********************100%%**********************]  13 of 13 completed


In [27]:
yahooDataFrame

Ticker,ABT,BIIB,CMCSA,DHR,GILD,MMM,NKE,ORCL,PFE,QCOM,T,TMO,WBA
Date,,,,,,,,,,,,,
2019-01-02,63.287037,304.690002,30.364082,87.207214,51.315907,122.182968,70.124199,41.368904,32.894070,50.268066,14.461870,216.677170,52.906776
2019-01-03,60.300270,307.000000,30.602617,84.708145,52.780006,117.582275,68.883827,40.966373,31.973791,48.779282,14.481455,209.517792,52.354687
2019-01-04,62.021305,318.329987,31.636251,87.163834,55.206669,122.419716,70.682869,42.732002,32.703934,49.567451,14.853523,218.987915,54.096481
2019-01-07,62.950108,322.399994,31.300545,87.519592,55.457424,122.138191,71.696030,43.408985,32.878853,49.427326,15.122788,221.940567,54.407513
2019-01-08,62.194317,324.440002,31.556740,89.749680,55.077255,122.650078,72.652336,43.802368,33.030972,48.998215,15.313722,227.105164,55.496124
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,135.071182,234.190002,47.100624,287.357208,66.417831,127.586395,163.100189,86.083389,53.233276,176.740417,15.800819,658.553162,44.110512
2021-12-28,134.125900,235.990005,47.137951,283.872833,66.508965,128.265121,161.971207,85.227768,52.163216,175.308151,15.826327,651.760925,44.719166
2021-12-29,134.813370,258.309998,47.212612,286.814911,67.119637,128.821091,164.268097,84.804756,51.776562,176.617142,15.711552,659.070312,45.432163


In [28]:
#set parameters to calculate predict market drop
market_drop_info = pd.read_excel("MarketDD.xlsx",index_col=0)
swan_to_predict=9
total_day=yahooDataFrame.shape[0]
t1=int(round(market_drop_info.loc[:end_swan,"T1"].mean(),0))
t1=int((np.array(market_drop_info.loc[:end_swan,"T1"])*np.array(weight_swan)).sum())
#t2=total_day-t1
t2=int(round(market_drop_info.loc[:end_swan,"T2"].mean(),0))
dd=market_drop_info.loc[:end_swan,"Actual DD"].mean()
t0=int(total_day-t1-t2)

In [29]:
#calculate predict market drop
md = [] #Make market drop data list named md


j = 0 #Start from 0
inc = dd/(t1-1) #Calculate unit increasing market drop
for i in range(t1): #Append market drop data for t1
    md.append(round(j,4))
    j = j + inc

base = (t2+1)**(1/(dd*100)) #Calculate log base
l1 = [] #Make decreasing market drop list
for i in range(t2):
    l1.append(math.log((i+2),base)/100)

for i in range(t2): #Append market drop data for t2
    md.append(round((dd-l1[i]),4))

for i in range(t0):
    md.append(0)

market_drop = pd.DataFrame(md) #Change list into dataframe
market_drop = market_drop.rename(columns={0: 'Swan Drop spy'}) #Change column name
market_drop1 = pd.read_csv("trainingMarketdrop.csv")

In [30]:
market_drop.to_csv("market_drop.csv")

In [31]:
market_drop

,Swan Drop spy
0,0.0000
1,0.0040
2,0.0080
3,0.0119
4,0.0159
...,...
752,0.0000
753,0.0000
754,0.0000
755,0.0000


In [32]:
swan_drop_pct=np.array(swan_beta)*np.array(market_drop)
yahooDataFrame=pd.DataFrame((1-swan_drop_pct)*np.array(yahooDataFrame),index=yahooDataFrame.index,columns=yahooDataFrame.columns)

In [33]:
yahooDataFrame

Ticker,ABT,BIIB,CMCSA,DHR,GILD,MMM,NKE,ORCL,PFE,QCOM,T,TMO,WBA
Date,,,,,,,,,,,,,
2019-01-02,63.287037,304.690002,30.364082,87.207214,51.315907,122.182968,70.124199,41.368904,32.894070,50.268066,14.461870,216.677170,52.906776
2019-01-03,60.173056,306.352329,30.538056,84.475898,52.635298,117.259896,68.694966,40.778480,31.814301,48.535963,14.388453,208.064098,51.991435
2019-01-04,61.759615,316.986839,31.502767,86.685874,54.903945,121.748432,70.295282,42.340018,32.377669,49.072951,14.662740,215.949116,53.345808
2019-01-07,62.555014,320.376517,31.104093,86.805725,55.005077,121.141952,71.111230,42.816672,32.390938,48.693835,14.833854,217.359406,53.284466
2019-01-08,61.672757,321.719258,31.292106,88.771553,54.477002,121.313390,71.860543,43.003786,32.376035,48.026681,14.922792,220.841679,53.965560
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,135.071182,234.190002,47.100624,287.357208,66.417831,127.586395,163.100189,86.083389,53.233276,176.740417,15.800819,658.553162,44.110512
2021-12-28,134.125900,235.990005,47.137951,283.872833,66.508965,128.265121,161.971207,85.227768,52.163216,175.308151,15.826327,651.760925,44.719166
2021-12-29,134.813370,258.309998,47.212612,286.814911,67.119637,128.821091,164.268097,84.804756,51.776562,176.617142,15.711552,659.070312,45.432163


In [34]:
yahooDataFrame=yahooDataFrame.dropna(how="all",axis = 1)
selected_stock_list=yahooDataFrame.columns.values
selected_stock_number = yahooDataFrame.shape[1]
selected_stock_list

array(['ABT', 'BIIB', 'CMCSA', 'DHR', 'GILD', 'MMM', 'NKE', 'ORCL', 'PFE',
       'QCOM', 'T', 'TMO', 'WBA'], dtype=object)

# Step3 - MVO

In [35]:
#Set the paramter for MVO process
Portfolio_Date=end

In [36]:
#data cleaning - filling missing data in yahooDataFrame
data_cleaning(yahooDataFrame)

In [37]:
bond_yield = quandl.get('FRED/DGS3MO')
bond_yield=np.log(bond_yield/100+1)
date=list(map(str,bond_yield.index.values))
date=list(map(modify_date,date))
week=pd.to_datetime(date).strftime("%W").values
week_1=np.roll(week,1)
week_difference=np.where(week!=week_1)
week_difference=np.delete(week_difference,0,None)
rf_week=bond_yield.iloc[week_difference-1]

In [38]:
# Calculated R for 1 year and 0.5 year
today_price = yahooDataFrame.iloc[-MA_days:,:]
one_year_ago_data = get_price_years_ago(yahooDataFrame,end,1)
one_year_ago_date = one_year_ago_data.index.values[-1]
half_year_ago_data = get_price_years_ago(yahooDataFrame,end,0.5)
half_year_ago_date = half_year_ago_data.index.values[-1]
one_year_ago_price=yahooDataFrame.loc[one_year_ago_date:,:]
one_year_ago_price=yahooDataFrame.iloc[:MA_days,:]
half_year_ago_price=yahooDataFrame.loc[half_year_ago_date:,:]
half_year_ago_price=yahooDataFrame.iloc[:MA_days,:]

R_mv_one_year=np.mean(pd.DataFrame(np.array(today_price)/np.array(one_year_ago_price)-1,index=today_price.index,columns=today_price.columns),axis=0)
R_mv_half_year=np.mean(pd.DataFrame(np.array(today_price)/np.array(half_year_ago_price)-1,index=today_price.index,columns=today_price.columns),axis=0)

In [39]:
R_mv_one_year.index

Index(['ABT', 'BIIB', 'CMCSA', 'DHR', 'GILD', 'MMM', 'NKE', 'ORCL', 'PFE',
       'QCOM', 'T', 'TMO', 'WBA'],
      dtype='object', name='Ticker')

In [40]:
# Calculate MA
MA_Data=yahooDataFrame.rolling(MA_days).mean().dropna(axis=0)
MA_Data=MA_Data.loc[one_year_ago_date:,:]
MA_now = MA_Data.iloc[-1,:]
MA_one_year_ago = MA_Data.iloc[0,:]
MA_half_year_ago = MA_Data.loc[half_year_ago_date,:]

In [41]:
# Calculate daily return
daily_return = yahooDataFrame.pct_change().dropna()
log_daily_return = np.log(1+daily_return)

# Get week end stock price data
stock_date = yahooDataFrame.index.values
week=pd.to_datetime(stock_date).strftime("%W").values
week_1=np.roll(week,1)
week_difference=np.where(week!=week_1)
week_difference=np.delete(week_difference,0,None)
# Calculate weekly return

weekly_return=yahooDataFrame.iloc[week_difference-1].pct_change().dropna()
log_weekly_return = np.log(1+weekly_return)

# Get month end stock price data
stock_date = yahooDataFrame.index.values
month=pd.to_datetime(stock_date).strftime("%m").values
month_1=np.roll(month,1)
month_difference=np.where(month!=month_1)
month_difference=np.delete(month_difference,0,None)
# Calculate monthly return
monthly_return=yahooDataFrame.iloc[month_difference-1].pct_change().dropna()
log_monthly_return = np.log(1+monthly_return)

# Get semi-month end stock price data
semi_month = []
for i in range(1,len(month_difference)):
    semi_month.append((month_difference[i]-month_difference[i-1])//2+month_difference[i-1])
    semi_month.append(month_difference[i]-1)
# Calculate semi-monthly return
semi_monthly_return=yahooDataFrame.iloc[semi_month].pct_change().dropna()
log_semi_monthly_return = np.log(1+semi_monthly_return)

In [42]:
# Calculated Max Dropdown for each stock
stock_return_one_year=daily_return.loc[one_year_ago_date:,:]
mdd=[]
for column in stock_return_one_year.columns:
    mdd .append(max_drawdown_absolute(stock_return_one_year.loc[:,column]))
mdd=pd.DataFrame(mdd,index = stock_return_one_year.columns,columns=["MAXDD","start_index","end_index"])
mar=R_mv_one_year/mdd.iloc[:,0]

In [43]:
#calculate weekly sd for each stock
one_year_weekly_return=log_weekly_return.iloc[log_weekly_return.index >= one_year_ago_date]
weekly_std= np.std(one_year_weekly_return)*np.sqrt(one_year_weekly_return.shape[0])

#Calculate garch_std sd for each stock
one_year_monthly_return=log_monthly_return.iloc[log_monthly_return.index >= one_year_ago_date]
res_all = one_year_monthly_return - np.mean(one_year_monthly_return)
judge = one_year_monthly_return.shape[0]
for stock in one_year_monthly_return.columns:
    res=res_all[stock]
    h= het_arch(res)
    if judge < 100 or h ==0:
        garch_std = weekly_std

    else:#build Garch Model
        std = []
        if judge <=200:
            p = 0
            q = 6
        else:
            p = 1
            q = 1

        std.append(fit_garch(one_year_monthly_return[stock],p,q))
        garch_std = pd.DataFrame(std,index = one_year_monthly_return.columns)

#Calculate monthly sd for each stock
one_year_monthly_return=log_monthly_return.iloc[log_monthly_return.index >= one_year_ago_date]
monthly_std = np.std(one_year_monthly_return)*np.sqrt(12);

#Calculate semi_monthly sd for each stock
one_year_semi_monthly_return=log_semi_monthly_return.iloc[log_semi_monthly_return.index >= one_year_ago_date]
semi_monthly_std = np.std(one_year_semi_monthly_return)*np.sqrt(24);


In [44]:
# Calculated MaxDrop
DD_range = 90
total_range = 90
DD_end = yahooDataFrame.iloc[yahooDataFrame.index <= one_year_ago_date]
DD_start = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(total_range)]
max_DD=[-1]*selected_stock_number


if (DD_start.shape[0] != 1 ) and (DD_end.shape[0] != 1):
    DD_start = DD_start.index[-1]
    DD_end = DD_end.index[-1]
    while DD_start > DD_end :
        DD_start = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(total_range)]
        DD_mid = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(DD_range+total_range)]
        if (DD_start.shape[0] != 1 ) and (DD_mid.shape[0] != 1):
            DD_start = DD_start.index[-1]
            DD_mid = DD_mid.index[-1]
            price = yahooDataFrame.loc[DD_start:DD_mid,:]
            for i in range(len(yahooDataFrame.columns)):
                max_DD[i]=max(max_DD[i],max_drawdown_absolute(stock_return_one_year.iloc[:,i])[0])
            total_range = total_range + 1
        else:
            break

In [46]:
optdata

,P_fwd,EPS_ttm,EPS_fwd,IsETF
Symbol,,,,
AAPL,306.8500,12.69,15.13,0
ABBV,93.4600,9.53,10.37,0
ABT,94.0700,3.25,3.82,0
ACN,186.0900,7.60,8.09,0
ADBE,338.5400,9.75,11.17,0
...,...,...,...,...
WBA,47.9400,5.70,5.97,0
WFC,36.9500,3.30,3.79,0
WMT,129.2600,5.13,5.45,0


In [45]:
# Read OPT-data
optdata = pd.read_excel("optdata.xlsx",index_col=0,sheet_name = 1)
P_current = MA_now
P_fwd = optdata.P_fwd.loc[P_current.index]
R_fwd=P_fwd/P_current-1
R_60 = P_current/MA_one_year_ago - 1
R_60_half = P_current/MA_half_year_ago - 1
R_MA_1ago = R_mv_one_year
R_MA_halfago = R_mv_half_year
Rfwd_adjusted = pd.Series(index=R_MA_1ago.index)
for i in R_fwd.index:
    if R_fwd[i] < 0.5*R_60[i]:
        Rfwd_adjusted[i] = R_60_half[i]
    elif R_fwd[i] > 1.3*R_60[i]:
        if 1.3*R_60[i] > 0.7*R_fwd[i]:
            Rfwd_adjusted[i] = 1.3*R_60[i]
        else:
            Rfwd_adjusted[i] = 0.7*R_fwd[i]
    else:
        Rfwd_adjusted[i] = R_fwd[i]
Rfwd_adjusted[Rfwd_adjusted<0]=0

KeyError: "['TMO'] not in index"

In [47]:
price_now = today_price.iloc[-1,:]
optimizer = optdata.loc[P_current.index,:]
FFC = pd.DataFrame([R_MA_1ago.values,R_MA_halfago.values,MA_now.values,price_now.values,semi_monthly_std.values,weekly_std.values,MA_one_year_ago.values,MA_half_year_ago.values,max_DD,R_60.values,R_60_half.values,P_fwd.values,Rfwd_adjusted.values],columns = R_MA_1ago.index,index = ["R_MA_1ago","R_MA_halfago","MA_now","price",'std','std_f','MA_1ago','MA_halfago','Fall_MDD','R_60','R_60_half','R_fwd',"Rfwd_adjusted"]).T
preselected_stock = preselected_stock.loc[selected_stock_list,:]
preselected_stock["Rfwd_adjusted"]=Rfwd_adjusted.values
preselected_stock["std_f"]=weekly_std.values
preselected_stock["SR"]=preselected_stock["Rfwd_adjusted"]/preselected_stock["std_f"]
preselected_stock["PR_SR"]=preselected_stock["PR"]*preselected_stock["SR"]**0.5
preselected_stock.to_csv("preselected_Stocks_info.csv")
p=FFC.MA_now
EPS_fwd=optimizer.EPS_fwd
EPS_ttm=optimizer.EPS_ttm
fall=FFC.Fall_MDD
# FFC.to_csv("WEDataBank.csv")

PE=p/EPS_fwd
PEG= p/(EPS_fwd*(EPS_fwd/EPS_ttm-1))
PEG[PEG<0]=2000

KeyError: "['TMO'] not in index"

In [234]:
preselected_stock

,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10a,avg,PR,Rfwd_adjusted,std_f,SR,PR_SR
Ticker,,,,,,,,,,,,,
AAPL,1.345472,0.858909,0.279703,1.582785,1.829198,0.972646,0.955996,1.117816,-0.117816,0.578740,0.239959,2.411830,-0.182968
ADBE,1.599366,1.393154,1.278684,0.687330,1.210366,0.628947,1.680628,1.211211,-0.211211,0.153273,0.302558,0.506590,-0.150329
AMZN,-0.445606,0.328167,0.123350,0.334033,1.561997,0.390752,1.440979,0.533381,0.466619,9.058544,0.241910,37.445897,2.855381
BAC,0.594106,2.142401,2.667373,1.184257,1.192921,1.509426,1.144357,1.490692,-0.490692,0.084898,0.246937,0.343806,-0.287717
BRK-B,0.220942,1.172896,0.533684,0.870710,0.663086,0.928445,-0.420873,0.566984,0.433016,0.017195,0.142329,0.120813,0.150508
COST,0.829818,0.976294,-0.015790,0.618966,0.918643,0.368083,0.536185,0.604600,0.395400,0.419931,0.191225,2.196008,0.585941
GOOGL,NaN,0.803565,1.167641,0.870361,0.938163,0.979057,0.880849,0.939939,0.060061,6.547853,0.242882,26.958963,0.311848
GS,1.029728,1.064879,1.959793,1.207118,1.557229,1.356818,1.301592,1.353880,-0.353880,0.052422,0.262662,0.199578,-0.158093
HD,1.546126,0.638987,0.756432,0.161429,1.098092,1.043489,1.605762,0.978617,0.021383,0.308801,0.232805,1.326433,0.024627


In [235]:
cov = pd.DataFrame(np.cov(np.exp(one_year_weekly_return)-1,rowvar=False,bias = False)*51,index= one_year_weekly_return.columns, columns= one_year_weekly_return.columns)
E_2 = Rfwd_adjusted/fall
E_3 = Rfwd_adjusted/PEG
E_4 = Rfwd_adjusted
std =  np.std(np.exp(one_year_weekly_return)-1)*np.sqrt(52)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/wenyiye/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


# Optimization

In [236]:
w0 = 1/selected_stock_number*np.ones(selected_stock_number)
uplim_ini = np.zeros(selected_stock_number)
uplim3 =pd.DataFrame( w1*(1-uplim_ini) + w2 * uplim_ini,index = std.index,columns=["weight"])
for i in uplim3.index:
    if i in ["SPY","IWD"]:
        uplim3.loc[i,:]=0
low_lim =  np.zeros(selected_stock_number)
multi=lambda x: x.T @cov@x

In [237]:
if method == "gptLowest8":
    cleaned_weights =  {'AMZN': 0.056, 'AAPL': 0.056, 'MSFT': 0.056, 'GOOGL': 0.056,
                  'NVDA': 0.056, 'NFLX': 0.056, 'MA': 0.056, 'V': 0.056,
                  'UNH': 0.056, 'JPM': 0.056, 'GS': 0.056, 'COST': 0.056,
                  'HD': 0.056, 'TGT': 0.056, 'ADBE': 0.056, 'BAC': 0.056,
                  'LMT': 0.056, 'BRK-B': 0.056}
    
else:
    ef = EfficientFrontier(E_2/100, cov,weight_bounds=(0, w1))
    #ef.add_constraint(lambda w: w@E_2== 12)
    raw_weights = ef.max_sharpe(0.01)
    cleaned_weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True)
    
print(cleaned_weights)

Expected annual return: 25.2%
Annual volatility: 11.5%
Sharpe Ratio: 2.11
OrderedDict([('AAPL', 0.0), ('ADBE', 0.0), ('AMZN', 0.17), ('BAC', 0.0), ('BRK-B', 0.14176), ('COST', 0.11802), ('GOOGL', 0.17), ('GS', 0.0), ('HD', 0.0), ('JPM', 0.09796), ('LMT', 0.11963), ('MA', 0.0), ('MSFT', 0.0), ('NFLX', 0.00478), ('NVDA', 0.0), ('TGT', 0.00784), ('UNH', 0.17), ('V', 0.0)])


/Users/wenyiye/anaconda3/lib/python3.11/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


In [238]:
weight1=np.array(pd.DataFrame(cleaned_weights.values(), index = list(cleaned_weights),columns = ["weight"]))[:,0]

In [239]:
sd1 = np.sqrt(weight1@cov@weight1.T)
R1 = weight1.T@E_2/100
sharpe_ratio1 = (R1-0.01)/sd1
print("new_sharpe:",sharpe_ratio1)

new_sharpe: 2.1096492834059033


weight2 =   pd.read_excel("old_weight.xlsx",index_col=0)
weight2=np.array(weight2.loc[list(cleaned_weights),:])[:,0]

sd2 = np.sqrt(weight2@cov@weight2.T)
R2 = weight2.T@E_2/100
sharpe_ratio2 = (R2-0.01)/sd2
print("old_sharpe:",sharpe_ratio2)

In [240]:
weight = pd.DataFrame(cleaned_weights.values(), index=list(cleaned_weights), columns=["weight"])
weight = weight[weight.weight >= 0.03]
weight.weight = weight.weight / sum(weight.weight)
weight.to_csv("weights.csv")

See pyportfolioopt.readthedocs.io/en/latest/MeanVariance.html for more reference


In [241]:
if stock_type == "S&P100":
    OEX=pdr.get_data_yahoo("^OEX", start=start_swan_date, end=end_swan_date)["Adj Close"]

elif stock_type == "S&P500":
    OEX=pdr.get_data_yahoo("SPY", start=start_swan_date, end=end_swan_date)["Adj Close"]

stock_price=pdr.get_data_yahoo(list(weight.index), start=start_swan_date, end=end_swan_date)["Adj Close"]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  7 of 7 completed


In [242]:
stock_price.to_csv("tradeinfo.csv")
stock_price

,AMZN,BRK-B,COST,GOOGL,JPM,LMT,UNH
Date,,,,,,,
2022-01-03,170.404495,300.790009,545.488037,144.991501,151.282242,336.178009,488.573212
2022-01-04,167.522003,308.529999,543.100952,144.399506,157.017334,343.416534,477.503723
2022-01-05,164.356995,309.920013,529.326660,137.774994,154.146729,339.764099,476.326782
2022-01-06,163.253998,313.220001,529.211243,137.747498,155.784393,339.631256,456.833618
2022-01-07,162.554001,319.779999,516.101196,137.016998,157.327927,341.661560,446.085205
...,...,...,...,...,...,...,...
2022-12-23,85.250000,306.489990,448.304230,89.230003,126.698761,470.532135,523.429626
2022-12-27,83.040001,305.549988,444.282928,87.389999,127.142708,472.683807,524.099609
2022-12-28,81.820000,303.429993,438.943726,86.019997,127.837593,470.464020,520.612122


In [243]:
start=stock_price.loc["2022-01-03"]
end = stock_price.loc["2022-10-12"]
stock_drop=abs(end/start-1)
market_drop = abs(OEX.loc["2022-10-12"]/OEX.loc["2022-01-03"]-1)

In [244]:
beta = stock_drop/market_drop
1-beta.T@weight

weight    0.248632
dtype: float64

In [245]:
stock_drop

AMZN     0.337459
BRK-B    0.122311
COST     0.173026
GOOGL    0.327133
JPM      0.339021
LMT      0.143832
UNH      0.005030
dtype: float64

In [246]:
print(stock_price.head(1))
weight

                  AMZN       BRK-B        COST       GOOGL         JPM  \
Date                                                                     
2022-01-03  170.404495  300.790009  545.488037  144.991501  151.282242   

                   LMT         UNH  
Date                                
2022-01-03  336.178009  488.573212  


,weight
AMZN,0.172175
BRK-B,0.143573
COST,0.119530
GOOGL,0.172175
JPM,0.099213
LMT,0.121160
UNH,0.172175


In [247]:
a=stock_price@weight
b=1-a.loc["2022-10-12"]/a.loc["2022-01-03"]
1-b/market_drop

weight    0.594123
dtype: float64

In [248]:
injected_result = pd.DataFrame(index = stock_price.index)
S1 = np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T


In [249]:
portfolio_value = np.sum(np.array(weight.T) * stock_price,axis=1)
S1 = np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T
injected_result["portfolio_value"]=stock_price@S1.T
S2 = 10000/float(OEX[0])
injected_result["oex"]=S2*OEX
injected_result.to_csv("Injected_result.csv")

/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/2729638000.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S2 = 10000/float(OEX[0])


In [251]:
temp=injected_result.loc[["2022-01-03","2022-10-12"]]
Portfolio_DD=abs(temp['portfolio_value'][1]/temp['portfolio_value'][0]-1)
DD=abs(temp['oex'][1]/temp['oex'][0]-1)
PR=1-(Portfolio_DD/DD)
print(PR)
temp
DD

0.3829905043103061


/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/2994947419.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Portfolio_DD=abs(temp['portfolio_value'][1]/temp['portfolio_value'][0]-1)
/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/2994947419.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  DD=abs(temp['oex'][1]/temp['oex'][0]-1)


0.2722984138075022

In [194]:
allocation=pd.read_excel('allocation weight.xlsx',index_col='stock selected')
dfMVOstock=deepcopy(weight)
dfMVOstock['PR_SR']=''
dfMVOstock['SR']=''
for i in range(len(weight)):
    dfMVOstock.iloc[i,1]=preselected_stock.loc[weight.index[i],'PR_SR']
    dfMVOstock.iloc[i,2]=preselected_stock.loc[weight.index[i],'SR']
dfMVOstock_sort=dfMVOstock.sort_values(by='PR_SR',ascending=False)
postallocationweight=post_allocation(dfMVOstock_sort)
postallocationweight


/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/2775426837.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (dfMVOweight.iloc[i,4]+residual/(numofstocks-i))>allocationweight[0]:
/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/2775426837.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfMVOweight.iloc[i,5]=allocationweight[0]
/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/2775426837.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFram

,weight,PR_SR,SR,equal,proportion,adjusted proportion,direct
AMZN,0.055556,2.855381,37.445897,0.055556,0.878472,0.07,0.07
LMT,0.055556,0.699408,1.190985,0.055556,0.215176,0.07,0.07
COST,0.055556,0.585941,2.196005,0.055556,0.180268,0.07,0.07
UNH,0.055556,0.382574,0.900578,0.055556,0.117701,0.07,0.07
GOOGL,0.055556,0.311848,26.958963,0.055556,0.095941,0.07,0.07
BRK-B,0.055556,0.150508,0.120813,0.055556,0.046305,0.07,0.07
MSFT,0.055556,0.095873,1.512674,0.055556,0.029496,0.07,0.07
TGT,0.055556,0.03069,0.086557,0.055556,0.009442,0.07,0.07
HD,0.055556,0.024627,1.326435,0.055556,0.007577,0.07,0.07
MA,0.055556,0.0,0.0,0.055556,0.0,0.07,0.07


In [125]:
#draw type E graph for postallocation
MVOweight=postallocationweight['weight']
equalweight=postallocationweight['equal']
proportionweight=postallocationweight['adjusted proportion']
directweight=postallocationweight['direct']
output(MVOweight).to_csv('no_post_allocation_result.csv')
output(equalweight).to_csv('equal_weight_result.csv')
output(proportionweight).to_csv('proportional_weight_result.csv')
output(directweight).to_csv('direct_weight_result.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  8 of 8 completed
[*********************100%%**********************]  1 of 1 completed


/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/1728796536.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S2 = 10000/float(OEX[0])


[*********************100%%**********************]  8 of 8 completed


/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/1728796536.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S2 = 10000/float(OEX[0])


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  8 of 8 completed
[*********************100%%**********************]  1 of 1 completed


/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/1728796536.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S2 = 10000/float(OEX[0])


[*********************100%%**********************]  8 of 8 completed


/var/folders/6g/__5vs7k9339d_y5ztlb880wm0000gn/T/ipykernel_46265/1728796536.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S2 = 10000/float(OEX[0])
